# Introduction to Scikit-Learn (sklearn)
This notebook demonstrates some of the most useful functions of the Scikit-Learn library.

Topics:



In [1]:
covering = [
    "0. end to end workflow",
    "1. getting the data ready",
    "2. choosing the right estimator / algorithm",
    "3. fitting the model (training)",
    "4. evaluating the model",
    "5. improving the model",
    "6. save and load a trained model",
    "7. putting it all together"
]
covering

['0. end to end workflow',
 '1. getting the data ready',
 '2. choosing the right estimator / algorithm',
 '3. fitting the model (training)',
 '4. evaluating the model',
 '5. improving the model',
 '6. save and load a trained model',
 '7. putting it all together']

In [2]:
# standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1. Getting the Data Ready

#### Three main things to do:
1. Split the data into features and labels (usually 'X' & 'y')
2. Filling (imputing) or disregarding missing values
3. Converting non-numerical values to numerical values (feature encoding)

In [3]:
heart = pd.read_csv("../data/heart-disease.csv")
heart.head(5)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
# splitting the data to X and y
# X represents the data to train on 
# y represents the expected results to train with
X = heart.drop("target", axis=1)
X.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2


In [5]:
y = heart["target"]
y.head()

0    1
1    1
2    1
3    1
4    1
Name: target, dtype: int64

In [6]:
# split the data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((242, 13), (61, 13), (242,), (61,))

In [7]:
# rows vs columns, original vs train set vs test set
X.shape, X_train.shape, X_test.shape

((303, 13), (242, 13), (61, 13))

### 1.1 Converting Data to all Numerical

All data must be numerical for the machine to understand. For the following example, we will convert the car_sales data to show "Make", "Colour", and "Doors" to be numerical values.

In [75]:
cars = pd.read_csv("../data/car-sales-extended.csv")
cars.head()

,Make,Colour,Odometer (KM),Doors,Price
0,Honda,White,35431,4,15323
1,BMW,Blue,192714,5,19943
2,Honda,White,84714,4,28343
3,Toyota,White,154365,4,13434
4,Nissan,Blue,181577,3,14043


In [76]:
cars.dtypes

Make             object
Colour           object
Odometer (KM)     int64
Doors             int64
Price             int64
dtype: object

In [77]:
# The goal is to have each value in the column assigned a numerical
# value that the machine can understand

# start by splitting
X = cars.drop("Price", axis=1)
y = cars["Price"]

X.head()

,Make,Colour,Odometer (KM),Doors
0,Honda,White,35431,4
1,BMW,Blue,192714,5
2,Honda,White,84714,4
3,Toyota,White,154365,4
4,Nissan,Blue,181577,3


#### Converting to Numerical and Viewing

In [78]:
# turn categories into numbers
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

categorical_features = ["Make", "Colour", "Doors"]
# note: doors are also considered catagorical, even tho they are
# already numerical, bc there are only three options for "Doors"

one_hot = OneHotEncoder()

# take the one_hot and apply it to the categorical_features
transformer = ColumnTransformer([("one_hot",
                                 one_hot,
                                 categorical_features)],
                                # ignore all the other columns
                               remainder="passthrough")

transformed_X = transformer.fit_transform(X)
transformed_X



array([[0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 3.54310e+04],
       [1.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        1.00000e+00, 1.92714e+05],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 8.47140e+04],
       ...,
       [0.00000e+00, 0.00000e+00, 1.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 6.66040e+04],
       [0.00000e+00, 1.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 2.15883e+05],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 1.00000e+00,
        0.00000e+00, 2.48360e+05]])

In [59]:
# looking at the below dataframe, we can see that:
# 4 "Makes" of cars
# 5 "Colours" of cars
# 3 "Doors" categories for cars

# Total of twelve. Each numbered column below represents one of these
# types of data, eg: col 0 is whether a car's "Make" is "BMW",
# '0' for 'no', '1', for 'yes'

transformed_X_df = pd.DataFrame(transformed_X)
transformed_X_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,35431.0
1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,192714.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,84714.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,154365.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,181577.0


In [60]:
cars["Make"].unique(),cars["Colour"].unique(),cars["Doors"].unique()

(array(['Honda', 'BMW', 'Toyota', 'Nissan'], dtype=object),
 array(['White', 'Blue', 'Red', 'Green', 'Black'], dtype=object),
 array([4, 5, 3], dtype=int64))

In [14]:
len(cars["Make"].unique()), len(cars["Colour"].unique()), len(cars["Doors"].unique())

(4, 5, 3)

In [15]:
# Another way to view it, in the original dataframe without making
# changes
dummies = pd.get_dummies(cars[["Make", "Colour", "Doors"]])
dummies.head()

,Doors,Make_BMW,Make_Honda,Make_Nissan,Make_Toyota,Colour_Black,Colour_Blue,Colour_Green,Colour_Red,Colour_White
0,4,0,1,0,0,0,0,0,0,1
1,5,1,0,0,0,0,1,0,0,0
2,4,0,1,0,0,0,0,0,0,1
3,4,0,0,0,1,0,0,0,0,1
4,3,0,0,1,0,0,1,0,0,0


#### Fitting the Model

In [16]:
# build the model
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()

np.random.seed(42)
# train and test split
X_train, X_test, y_train, y_test = train_test_split(transformed_X,
                                                   y,
                                                   test_size=0.2)
# train the model to learn the relationship between X and y data
model.fit(X_train, y_train)

RandomForestRegressor()

In [17]:
# score the model with a test
model.score(X_test, y_test)

0.3235867221569877

In [18]:
# Conclusion:
# the model worked; however, trying to predict a car's price
# based on the limited information in the table is likely not possible

# 32.35% success on the test is pretty dismal.


### 1.2 Handling Missing Values

**Options**
1. Fill with some value (imputation)
2. Remove the samples with missing values



In [79]:
# import a data set to work with
cars_m = pd.read_csv("../data/car-sales-extended-missing-data.csv")
cars_m.tail()

,Make,Colour,Odometer (KM),Doors,Price
995,Toyota,Black,35820.0,4.0,32042.0
996,NaN,White,155144.0,3.0,5716.0
997,Nissan,Blue,66604.0,4.0,31570.0
998,Honda,White,215883.0,4.0,4001.0
999,Toyota,Blue,248360.0,4.0,12732.0


In [80]:
cars_m.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

#### Using Pandas for Missing Values

In [81]:
# fill NaN values within feature columns
cars_m["Make"].fillna("missing", inplace=True)

cars_m["Colour"].fillna("missing", inplace=True)

cars_m["Odometer (KM)"].fillna(cars_m["Odometer (KM)"].mean(), inplace=True)

cars_m["Doors"].fillna(4, inplace=True)

In [82]:
cars_m.isna().sum()

Make              0
Colour            0
Odometer (KM)     0
Doors             0
Price            50
dtype: int64

In [83]:
# Price is our target column, so we'll just remove the rows that have
# missing Price values
cars_m.dropna(inplace=True)
cars_m.isna().sum(), len(cars_m)

(Make             0
 Colour           0
 Odometer (KM)    0
 Doors            0
 Price            0
 dtype: int64,
 950)

In [84]:
X = cars_m.drop("Price", axis = 1)
y = cars_m["Price"]

X["Make"].unique(), X["Colour"].unique(), X["Doors"].unique()

(array(['Honda', 'BMW', 'Toyota', 'Nissan', 'missing'], dtype=object),
 array(['White', 'Blue', 'Red', 'Green', 'missing', 'Black'], dtype=object),
 array([4., 5., 3.]))

In [85]:
X.head()

,Make,Colour,Odometer (KM),Doors
0,Honda,White,35431.0,4.0
1,BMW,Blue,192714.0,5.0
2,Honda,White,84714.0,4.0
3,Toyota,White,154365.0,4.0
4,Nissan,Blue,181577.0,3.0


In [88]:
# turn categories into numbers
categorical_features = ["Make", "Colour", "Doors"]

one_hot = OneHotEncoder()

transformer = ColumnTransformer([("one_hot",
                                 one_hot,
                                 categorical_features)],
                               remainder="passthrough")

transformed_X = transformer.fit_transform(X)
transformed_X

# NOTE: this saves the large object into a 'sparse matrix', which is a condensed
# type of array...

<950x15 sparse matrix of type '<class 'numpy.float64'>'
	with 3800 stored elements in Compressed Sparse Row format>

In [91]:
# trying to view the data gives the following...
transformed_X_df = pd.DataFrame(transformed_X)
transformed_X_df.head()

,0
0,"(0, 1)\t1.0\n (0, 9)\t1.0\n (0, 12)\t1.0\n..."
1,"(0, 0)\t1.0\n (0, 6)\t1.0\n (0, 13)\t1.0\n..."
2,"(0, 1)\t1.0\n (0, 9)\t1.0\n (0, 12)\t1.0\n..."
3,"(0, 3)\t1.0\n (0, 9)\t1.0\n (0, 12)\t1.0\n..."
4,"(0, 2)\t1.0\n (0, 6)\t1.0\n (0, 11)\t1.0\n..."


In [92]:
# to view the actual data as a dataFrame similar to way above,
# use the '.toarray()' on the compressed array
transformed_X_df = pd.DataFrame(transformed_X.toarray())
transformed_X_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,35431.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,192714.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,84714.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,154365.0
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,181577.0


In [94]:
# show the total amount of columns in the dataframe
print(list(transformed_X_df))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


#### Use SciKit Learn to fill missing data values

In [97]:
# NOTE: it's best to fill and transform training and test 
# sets separately

cars_m = pd.read_csv("../data/car-sales-extended-missing-data.csv")
cars_m.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

In [98]:
cars_m.dropna(subset=["Price"], inplace=True)
cars_m.isna().sum()

Make             47
Colour           46
Odometer (KM)    48
Doors            47
Price             0
dtype: int64

In [99]:
# Split into X and y
X = cars_m.drop("Price", axis=1)
y = cars_m["Price"]

In [101]:
# fill missing values with sklearn
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

# Fill catagorical values with 'missing' and numerical values with
# mean()

# if 'strategy' is 'constant', then a 'fill_value' needs to be supplied
cat_imputer = SimpleImputer(strategy="constant", fill_value="missing")
door_imputer = SimpleImputer(strategy="constant", fill_value=4)
num_imputer = SimpleImputer(strategy="mean")

# Define columnes
cat_features = ["Make", "Colour"]
door_features = ["Doors"]
num_features = ["Odometer (KM)"]

# Create imputer (fills in missing data)
imputer = ColumnTransformer([
    ("cat_imputer", cat_imputer, cat_features),
    ("door_imputer", door_imputer, door_features),
    ("num_imputer", num_imputer, num_features)
])

# transform the data
filled_X = imputer.fit_transform(X)
filled_X

array([['Honda', 'White', 4.0, 35431.0],
       ['BMW', 'Blue', 5.0, 192714.0],
       ['Honda', 'White', 4.0, 84714.0],
       ...,
       ['Nissan', 'Blue', 4.0, 66604.0],
       ['Honda', 'White', 4.0, 215883.0],
       ['Toyota', 'Blue', 4.0, 248360.0]], dtype=object)

In [105]:
# re-adding the column heads to the df
cars_m_filled = pd.DataFrame(filled_X,
                            columns=["Make","Colour",
                                     "Doors","Odometer (KM)"])
cars_m_filled.head()

,Make,Colour,Doors,Odometer (KM)
0,Honda,White,4.0,35431.0
1,BMW,Blue,5.0,192714.0
2,Honda,White,4.0,84714.0
3,Toyota,White,4.0,154365.0
4,Nissan,Blue,3.0,181577.0


In [106]:
# checking
cars_m_filled.isna().sum()

Make             0
Colour           0
Doors            0
Odometer (KM)    0
dtype: int64

In [108]:
# turn categories into numbers
categorical_features = ["Make", "Colour", "Doors"]

one_hot = OneHotEncoder()

transformer = ColumnTransformer([("one_hot",
                                 one_hot,
                                 categorical_features)],
                               remainder="passthrough")

transformed_X = transformer.fit_transform(cars_m_filled)
transformed_X

<950x15 sparse matrix of type '<class 'numpy.float64'>'
	with 3800 stored elements in Compressed Sparse Row format>

In [110]:
pd.DataFrame(transformed_X.toarray()).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,35431.0
1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,192714.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,84714.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,154365.0
4,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,181577.0


In [115]:
np.random.seed(42)
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(transformed_X,
                                                   y,
                                                   test_size=0.2)
model = RandomForestRegressor(n_estimators=100)
model.fit(X_train, y_train)
model.score(X_test, y_test)

0.21990196728583944

In [117]:
# sample difference can account for difference in scores
len(cars), len(cars_m)

(1000, 950)

## 2. Choosing the Right Model

Somethings to note:

* sklearn refers to 'machine learning models' and 'algorithms' as 'estimators.
* Classification problem - predicting a category (heart disease or not)
    * Sometimes we will see `clf` for classification model
* Regression problem - predicting a number (selling price)

<img src="../img/ml_map.png" />

https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html

### 2.1 Picking a machine learning model for regression

Practice datasets:
    https://scikit-learn.org/stable/datasets/toy_dataset.html

        

#### Import from the sklearn datasets

In [119]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

#### Convert the import into a dataframe

In [122]:
housing_df = pd.DataFrame(housing["data"], columns=housing["feature_names"])
housing_df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25
...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32


In [124]:
housing_df["MedHouseVal"]=housing["target"]
housing_df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [125]:
# With this dataset, we want to try to predict the MedHouseVal
# According to the flow chart above, we should be in the 'regression'
# bubble with <100k samples, but beyond that we're not sure
# So, we can experiment a bit

from sklearn.linear_model import Ridge


X = housing_df.drop("MedHouseVal", axis=1)
y = housing_df["MedHouseVal"] # median house price in $100,000's

# split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Instantiate and fit the model
model = Ridge()
model.fit(X_train, y_train)

# check score on test set
model.score(X_test, y_test) # returns 'coefficient of determination'
# i.e. how predicitive are the features on the target value?
# how to improve?
# add more data, or
# try a different model

0.6017138339794649

In [128]:
# trying another model(s)

from sklearn.linear_model import LassoLars
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression

X = housing_df.drop("MedHouseVal", axis=1)
y = housing_df["MedHouseVal"] # median house price in $100,000's

# split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# model = LassoLars()
# model = Lasso()
model = LinearRegression()
model.fit(X_train, y_train)

model.score(X_train, y_train)

0.6062594149243856